In [1]:
!pip install psycopg2

In [2]:
import pandas
import configparser
import psycopg2
from collections import Counter

In [3]:
config = configparser.ConfigParser()
config.read('config.ini')

host = config['myaws']['host']
user = config['myaws']['user']
pwd = config['myaws']['pwd']
database = config['myaws']['database']

In [4]:
conn = psycopg2.connect(
                        host = host,
                        user = user,
                        password = pwd,
                        dbname = database
)

In [5]:
cur = conn.cursor()

In [6]:
table_sql = """
            CREATE TABLE IF NOT EXISTS indeed_gp1a
            (
                id SERIAL,
                job_title VARCHAR(200),
                job_company VARCHAR(200),
                job_loc VARCHAR(200),
                job_salary VARCHAR(200),
                job_summary TEXT,
                PRIMARY KEY(id)
            );

            """

In [7]:
cur.execute(table_sql)
conn.commit()

In [8]:
url = 'https://www.indeed.com/jobs?q=intelligence+analyst&start=0'

In [9]:
import urllib.request
response = urllib.request.urlopen(url)
html_data= response.read()
#print(html_data.decode('utf-8'))


In [10]:
!pip install beautifulsoup4

In [11]:
from bs4 import BeautifulSoup
soup = BeautifulSoup(html_data,'html.parser')
#print (soup)

In [12]:
for table_resultsBody in soup.find_all('table', id = 'resultsBody'):
    pass
    #print(table_resultsBody)

In [13]:
for table_pageContent in table_resultsBody.find_all('table', id = 'pageContent'):
    pass
    #print(table_pageContent)

In [14]:
for td_resultsCol in table_pageContent.find_all('td', id = 'resultsCol'):
    pass
    #print(td_resultsCol)

In [15]:
#  identify the job title, company, ratings, reviews, salary, and summary
for div_row in td_resultsCol.find_all('div', class_='jobsearch-SerpJobCard unifiedRow row result'):

    # find job title
    job_title = None
    job_company  = None
    job_rating = None
    job_loc = None
    job_salary = None
    job_summary = None
    for div_title in div_row.find_all('div', class_ = 'title'):
        job_title = div_title.text.strip().replace("'","_")
    
    for div_dsc in div_row.find_all('div', class_ = 'sjcl'):
        
        #find company name
        for span_company in div_dsc.find_all('span', class_ = 'company'):
            job_company = span_company.text.strip().replace("'","_")
        
        # find location
        for div_loc in div_dsc.find_all('div', class_ = 'location accessible-contrast-color-location'):
            job_loc = div_loc.text.strip().replace("'","_")
    
    # find salary
    for div_salary in div_row.find_all('div',class_ ='salarySnippet'):
        job_salary = div_salary.text.strip().replace("'","_")
    
    #find summary
    for div_summary in div_row.find_all('div', class_ = 'summary'):
        job_summary = div_summary.text.strip().replace("'","_")
        
    # insert into database
    
    sql_insert = """
                insert into indeed_gp1a(job_title,job_company,job_loc,job_salary,job_summary)
                values('{}','{}','{}','{}','{}')            
                """.format(job_title,job_company,job_loc,job_salary,job_summary)

    cur.execute(sql_insert)
    conn.commit()

In [16]:
df = pandas.read_sql_query('select * from indeed_gp1a', conn)
df[:]

,id,job_title,job_company,job_loc,job_salary,job_summary
0,1,Intelligence Analyst,National Security Agency,"Fort Meade, MD","$47,016 - $74,759 a year",The National Security Agency (NSA) is part of ...
1,2,Associate Cyber Intelligence Analyst,LookingGlass Cyber Solutions,"Reston, VA 20191",None,"Past experience in intelligence, technology, c..."
2,3,"Cybersecurity Analyst (DLP) (60-70k, US Citize...",Maveris,"Martinsburg, WV","$60,000 - $70,000 a year",The Analyst can identify risks in security sys...
3,4,ERP Applications Developer (Epicor),"Ashworth Bros., Inc.","Winchester, VA 22601",None,This may include custom mobile and windows app...
4,5,Transportation Analyst,Lipman Family Farms,"Immokalee, FL 34142",None,"Perform fact based, data driven analysis, and ..."
5,6,Security Analyst,Seen by Indeed,"Washington, DC",None,Get matched – Companies apply to you with oppo...
6,7,Intelligence Analyst,Booz Allen Hamilton,None,None,We’re an EOE that empowers our people—no matte...
7,8,Intelligence Analyst I - MRS,WorldAware,None,None,WorldAware is dedicated to equal opportunity i...
8,9,Intelligence Analyst,BAE Systems,None,None,Having an understanding of physical and techni...
9,10,Global Intelligence & Travel Security Operatio...,Pinkerton,None,None,The Global Intelligence & Travel Security Oper...


In [17]:
sql_statement = '''
                select job_summary from indeed_gp1a
                '''

In [18]:
cur.execute(sql_statement)
summaries = cur.fetchall()
print(summaries)


[('The National Security Agency (NSA) is part of the DoD Intelligence Community Defense Civilian Intelligence Personnel System (DCIPS).',), ('Past experience in intelligence, technology, cyber-security is preferred. Familiarity with threats, fraud, information technology vulnerabilities and brand…',), ('The Analyst can identify risks in security systems and work with technical experts to resolve security issues. Must be able to obtain a Public Trust clearance.',), ('This may include custom mobile and windows apps, IOT (Internet of Things) and PLC monitoring applications, API/REST/ODATA application and database integrations,…',), ('Perform fact based, data driven analysis, and modeling on significant amounts of data across the supply chain for the purpose of turning it into actionable…',), ('Get matched – Companies apply to you with opportunities that reflect your role, location and salary specifications. Get matched with top tech companies on Seen!',), ('We’re an EOE that empowers our 

In [19]:
x = list(map(''.join, summaries))
summary = []
word = ""
for a in range(len(x)):
    words = x[a].split()
    for b in range(len(words)):
        word = words[b].strip(',')
        word = word.strip('.')
        word = word.strip('...')
        summary.append(word)

In [20]:
counter_result = Counter(summary)
for word, count in counter_result.most_common(50):
    print(word,count)

and 213
to 119
the 87
of 84
is 69
in 58
The 49
intelligence 49
Intelligence 47
technical 45
on 44
national 44
or 42
provide 36
race 33
religion 33
security 32
a 31
analysis 29
an 28
experience 27
with 26
for 25
sexual 23
color 22
sex 22
other 22
orientation 20
origin 20
employment 20
without 20
regard 20
analyze 20
assessments 20
information 19
our 19
as 19
Analyst 18
issues 18
gender 18
that 17
science 17
Community 16
threat 16
Engineering 16
students 16
those 16
programs 16
written 16
oral 16


In [21]:
cur.close()
conn.close()

In [23]:
import xlwt
mybook = xlwt.Workbook()
sheet = mybook.add_sheet('summary_word_count')
i=0
sheet.write(0,0,'word')
sheet.write(0,1,'count')
for word, count in counter_result.most_common(50):
    i =i+1
    sheet.write(i,0,word)
    sheet.write(i,1,count)
    
mybook.save('summary_word_count.xls')

<img src='word_cloud.jpg'>